In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [2]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [3]:
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [4]:
columns_sorted = files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True) [0]
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [7]:
def count_move_small_tables(table):
    df = pd.read_json(os.path.join(inputdir+table), compression='gzip', lines=True)
    if len(df)<=10:
        df.to_json(os.path.join(outputdir+table), orient = 'records', compression = 'gzip', lines = True)
        print(len(df))
        return
    row_df = pd.DataFrame ()
    row_df = row_df.append({'file': table, 'length': len(df)}, ignore_index=True)

In [ ]:
row_list = list()
for file in files:
    #print(index)
    input = '../../src/data/data/'+index+'/All/'
    output = '../../src/data/data/'+index
    if not os.path.exists(output):
        os.makedirs(output)
    #for _ in tqdm(count_move_small_tables, [file for file in os.columns_sorted(input) if file.endswith('.json.gz')]), total=len([file for file in os.columns_sortedinput) if file.endswith('.json.gz')]):
     #   pass
    df = pd.read_json(os.path.join(input, '{}'.format(file)), compression='gzip', lines=True)
    if len(df) <= 10:
        df.to_json(os.path.join(output+'/under_10_rows/'+file), compression = 'gzip', lines = True)
        print(len(df))
    elif len(df)>10:
        row_list.extend([table,len(df)])
        print(len([file for file in os.listdir(output) if file.endswith('.json.gz')]))

In [5]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/Book/'

In [6]:
files = [file for file in os.listdir(path) if file.endswith('.json.gz')]

In [ ]:
row_df = pd.DataFrame ()
for file in files:
        df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
        if len(df) <= 10:
           df.to_json(os.path.join(path+'/under_10_rows/'+file), compression = 'gzip', lines = True)
        print(len(df))

        row_df = row_df.append({'file': file, 'length': len(df)}, ignore_index=True)
        #print(row_list)
   
          
          #os.remove(os.path.join(output, '{}'.format(file)))

print(file, len(files))
print(row_df)

In [8]:
#files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
       
if __name__ == '__main__':
    for index in columns_sorted.filename:
        print(index)
        setcol = set(columns_sorted.loc[index]['listofcolumns'])
        print(setcol)
        inputdir = '../../src/data/schemafiltereddata/'+index+'/'
        outputdir = '../../src/data/schemafiltereddata/'+index+'/under_10_rows/'
        if not os.path.exists(outputdir):
            os.makedirs(outputdir)
        p = Pool(100)
        for _ in tqdm(p.imap_unordered(count_move_small_tables, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]), total=len([file for file in os.listdir(inputdir) if file.endswith('.json.gz')])):
            pass
        p.map(count_move_small_tables, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')])
        p.close()
        print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
        p.join()

Book
{'publisher', 'isbn', 'datepublished', 'bookedition', 'offers', 'inlanguage', 'genre', 'name', 'aggregaterating', 'bookformat', 'author', 'numberofpages'}


100%|██████████| 978/978 [00:05<00:00, 191.58it/s]


0
CreativeWork
{'mainentityofpage', 'inlanguage', 'genre', 'version', 'name', 'aggregaterating', 'interactioncount', 'commentcount', 'keywords', 'articlebody', 'datemodified', 'datepublished', 'offers', 'headline', 'publisher', 'author', 'copyrightyear', 'creator', 'interactiontype', 'copyrightholder', 'interactionstatistic', 'datecreated', 'alternativeheadline'}


 20%|█▉        | 1583/7933 [00:01<00:07, 877.05it/s]

538

3

56

...

0
Restaurant
{'openinghours', 'openinghoursspecification', 'hasmenu', 'acceptsreservations', 'telephone', 'servescuisine', 'menu', 'pricerange', 'name', 'aggregaterating', 'geo', 'address'}


100%|██████████| 734/734 [00:03<00:00, 235.01it/s]


0
TVEpisode
{'episodenumber', 'name', 'partofseries'}


100%|██████████| 210/210 [00:02<00:00, 81.39it/s] 


0


In [19]:
row_df

,file,length
0,Book_foldvaribooks.com_September2020.json.gz,274.0
1,Book_virago.co.uk_September2020.json.gz,398.0
2,Book_borgantiquarian.com_September2020.json.gz,248.0
3,Book_downpour.com_September2020.json.gz,12194.0
4,Book_marasworld.com_September2020.json.gz,26.0
...,...,...
973,Book_benjamins.com_September2020.json.gz,1795.0
974,Book_nollegiu.com_September2020.json.gz,3175.0
975,Book_nosegraze.com_September2020.json.gz,153.0
976,Book_hachettebooks.com_September2020.json.gz,583.0


In [22]:
row_df.mean(axis = 1).mean()

1512.4744376278118